<a href="https://colab.research.google.com/github/VenkataBhanuTejaKonijeti/A-Semantic-Focused-Web-Crawler-Using-Sentence-BERT-and-Priority-Based-Crawling/blob/main/with_seed_urls.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install sentence-transformers beautifulsoup4 requests nltk


In [ ]:
import requests
import heapq
import re
import nltk
import numpy as np
import time
from bs4 import BeautifulSoup
from sentence_transformers import SentenceTransformer
from nltk.corpus import stopwords
from urllib.parse import urljoin, urlparse


In [ ]:
nltk.download('stopwords')
STOP_WORDS = set(stopwords.words('english'))


In [ ]:
model = SentenceTransformer("all-MiniLM-L6-v2")


In [ ]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r'http\S+', '', text)
    text = re.sub(r'[^a-z\s]', '', text)
    words = text.split()
    words = [w for w in words if w not in STOP_WORDS]
    return " ".join(words)


In [ ]:
def fetch_page(url):
    try:
        headers = {"User-Agent": "Mozilla/5.0"}
        response = requests.get(url, headers=headers, timeout=10)

        if response.status_code != 200:
            return None, []

        soup = BeautifulSoup(response.text, "html.parser")

        # Remove unwanted elements
        for tag in soup(["script", "style", "noscript"]):
            tag.decompose()

        text = soup.get_text(separator=" ")
        text = clean_text(text)

        links = []
        for a in soup.find_all("a", href=True):
            link = urljoin(url, a["href"])
            parsed = urlparse(link)
            if parsed.scheme in ["http", "https"]:
                links.append(link)

        return text, links

    except:
        return None, []


In [ ]:
def semantic_similarity(query_embedding, document_text):
    doc_embedding = model.encode(
        document_text,
        normalize_embeddings=True
    )
    return float(np.dot(query_embedding, doc_embedding))


In [ ]:
def semantic_crawler(
    seed_urls,
    query,
    max_pages=20,
    relevance_threshold=0.35,
    delay=1
):
    visited = set()
    results = []

    query_embedding = model.encode(
        query,
        normalize_embeddings=True
    )

    # Priority queue (max-heap using negative score)
    frontier = []
    for url in seed_urls:
        heapq.heappush(frontier, (-1.0, url))

    while frontier and len(results) < max_pages:
        _, url = heapq.heappop(frontier)

        if url in visited:
            continue
        visited.add(url)

        print(f"🔍 Fetching LIVE URL: {url}")

        text, links = fetch_page(url)
        if not text:
            continue

        score = semantic_similarity(query_embedding, text)

        if score >= relevance_threshold:
            results.append({
                "url": url,
                "score": round(score, 4)
            })

            # Add outgoing links with priority
            for link in links:
                if link not in visited:
                    heapq.heappush(frontier, (-score, link))

        time.sleep(delay)

    return sorted(results, key=lambda x: x["score"], reverse=True)


In [ ]:
QUERY = "covid vaccine pandemic"

SEED_URLS = [
    "https://www.who.int",
    "https://www.cdc.gov",
    "https://www.nih.gov"
]


In [ ]:
results = semantic_crawler(
    seed_urls=SEED_URLS,
    query=QUERY,
    max_pages=20,
    relevance_threshold=0.35,
    delay=1
)


In [ ]:
print("\n📌 FINAL RANKED LIVE URL RESULTS\n")

for i, r in enumerate(results, 1):
    print(f"{i}. Score = {r['score']} | {r['url']}")
